# Проект по А/B-тестированию

# Цели исследования:
- Провести оценку результатов A/B-теста.
- Оценить корректность проведения теста и проанализируйте его результаты.\

# Описание проекта
В распоряжении есть датасет с действиями пользователей, техническое задание и несколько вспомогательных датасетов.
Необходимо оценить корректность проведения теста и проанализировать его результаты.

# Описание данных:

**/datasets/ab_project_marketing_events.csv** — календарь маркетинговых событий на 2020 год;\
Структура файла: \
- name — название маркетингового события;
- regions — регионы, в которых будет проводиться рекламная кампания;
- start_dt — дата начала кампании;
- finish_dt — дата завершения кампании.

**/datasets/final_ab_new_users.csv** — все пользователи, зарегистрировавшиеся в интернет-магазине в период с 7 по 21 декабря 2020 года;\
Структура файла:
- user_id — идентификатор пользователя;
- first_date — дата регистрации;
- region — регион пользователя;
- device — устройство, с которого происходила регистрация.

**/datasets/final_ab_events.csv** — все события новых пользователей в период с 7 декабря 2020 по 4 января 2021 года;\
Структура файла:
- user_id — идентификатор пользователя;
- event_dt — дата и время события;
- event_name — тип события;
- details — дополнительные данные о событии. Например, для покупок, purchase, в этом поле хранится стоимость покупки в долларах.

**/datasets/final_ab_participants.csv** — таблица участников тестов.\
Структура файла:
- user_id — идентификатор пользователя;
- ab_test — название теста;
- group — группа пользователя.

# Техническое задание:
Название теста: **recommender_system_test**;\
Группы: 
- А (контрольная), 
- B (новая платёжная воронка);
Дата запуска: 2020-12-07\
Дата остановки набора новых пользователей: 2020-12-21;\
Дата остановки: 2021-01-04;

Ожидаемое количество участников теста: 15% новых пользователей из региона EU;

Назначение теста: тестирование изменений, связанных с внедрением улучшенной рекомендательной системы;


Ожидаемый эффект: за 14 дней с момента регистрации в системе пользователи покажут улучшение каждой метрики не менее, чем на 5 процентных пунктов:

- конверсии в просмотр карточек товаров — событие product_page
- просмотры корзины — product_cart
- покупки — purchase.

Загрузите данные теста, проверьте корректность его проведения и проанализируйте полученные результаты.

# План работы

- Опишите цели исследования
- Исследуйте данные:
- Требуется ли преобразование типов?
- Опишите природу пропущенных значений и дубликатов, если их обнаружите.
- Оцените корректность проведения теста. Обратите внимание на:
- Соответствие данных требованиям технического задания. Проверьте корректность всех пунктов технического задания.
- Время проведения теста. Убедитесь, что оно не совпадает с маркетинговыми и другими активностями.
- Аудиторию теста. Удостоверьтесь, что нет пересечений с конкурирующим тестом и нет пользователей, участвующих в двух группах теста одновременно. Проверьте равномерность распределения по тестовым группам и правильность их формирования.
- Проведите исследовательский анализ данных:
- Количество событий на пользователя одинаково распределены в выборках?
- Как число событий в выборках распределено по дням?
- Как меняется конверсия в воронке в выборках на разных этапах?
- Какие особенности данных нужно учесть, прежде чем приступать к А/В- тестированию?
- Оцените результаты А/В-тестирования
- Что можно сказать про результаты А/В-тестирования?
- Проверьте статистическую разницу долей z-критерием.
- Опишите выводы по этапу исследовательского анализа данных и по проведённой оценке результатов А/В-тестирования. Сделайте общее заключение о корректности проведения теста.

## Загрузка данных. 

In [1]:
# Импотрируем необходимые билиотеки:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats as st
import math as mth
from scipy.stats import ttest_ind
from datetime import datetime 
from datetime import timedelta
from plotly.subplots import make_subplots

In [2]:
# Загружаем данные в переменные:
marketing_events = pd.read_csv('/datasets/ab_project_marketing_events.csv')
marketing_events.head(5)

FileNotFoundError: [Errno 2] No such file or directory: '/datasets/ab_project_marketing_events.csv'

In [ ]:
new_users = pd.read_csv('/datasets/final_ab_new_users.csv')
new_users.head(5)

In [ ]:
events = pd.read_csv('/datasets/final_ab_events.csv')
events.head(5)

In [ ]:
participants = pd.read_csv('/datasets/final_ab_participants.csv')
participants.head(5)

## Изучение и предобработка данных:

Подготовьте данные, оцените их целостность:
Требуется ли преобразование типов?
Присутствуют ли пропущенные значения и дубликаты? Если да, то какова их природа?

In [ ]:
# Напишем функцию для исследования данных в датафрейме:
def df_review(df):
    print('ТАБЛИЦА С ДАННЫМИ:')
    display(df.head(10))
    print('----------')
    print('INFO:')
    print(df.info())
    print('----------')
    print('ПРОПУСКИ В ДАТАФРЕЙМЕ')
    print(df.isna().sum())
    print('----------')
    print('ДУБЛИКАТЫ В ДАТАФРЕЙМЕ')
    print(df.duplicated().sum())
    print('----------')
    print('В датафрейме содержится строк\столбцов:')
    print(df.shape)
    print('----------')

### marketing_events
Датафрейм содержит календарь маркетинговых событий на 2020 год

In [ ]:
df_review(marketing_events)

Файл /datasets/ab_project_marketing_events.csv** — календарь маркетинговых событий на 2020 год:
- Необходимо преобразование типов object -> datetime64 в столбцах ['start_dt'], ['finish_dt'] 

In [ ]:
# Выполним необходимые преобразования типов:
marketing_events['start_dt'] = pd.to_datetime(marketing_events['start_dt'])
marketing_events['finish_dt'] = pd.to_datetime(marketing_events['finish_dt'])
marketing_events.info()

По датафрейму **marketing_events** выяснили следующую информацию:
- Датафрейм содержит 14 строк и 4 столбца
- Пропусков и полных дубликатов не найдено
- Выполнили преобразование типов object -> datetime64 в столбцах ['start_dt'], ['finish_dt'] 

### new_users
Датафрейм содержит всех пользователей, зарегистрировавшихся в интернет-магазине в период с 7 по 21 декабря 2020 года

In [ ]:
df_review(new_users)

In [ ]:
# Выполним необходимые преобразования типов:
new_users['first_date'] = pd.to_datetime(new_users['first_date'])
new_users.info()

In [ ]:
new_users['region'].unique()

Всего существует 5 уникальных регионов пользователей в датафрейме new_users.

Датафрейм должен содержать всех пользователей, зарегистрировавшихся в интернет-магазине в период с 7 по 21 декабря 2020 года,\
проверим, так ли это.

In [ ]:
#print('Минимальная дата регистрации', new_users['first_date'].min())
#print('Максимальная дата регистрации', new_users['first_date'].max())

Есть лишние данные. Уберем лишние даты регистрации, затем проверим результат.

In [ ]:
#new_users = new_users[new_users['first_date'] < '2020-12-22']
#print('Минимальная дата регистрации', new_users['first_date'].min())
#print('Максимальная дата регистрации', new_users['first_date'].max())

In [ ]:
#Сохраним всех пользователей из recommender_system_test:
check_tz = participants[participants['ab_test']=='recommender_system_test']
#check_tz
#Приссоединим к эти пользователям таблицу new_users:
check_tz = check_tz.merge(new_users)
check_tz.head(5)

In [ ]:
print('Минимальная дата регистрации', check_tz['first_date'].min())
print('Максимальная дата регистрации', check_tz['first_date'].max())

По датафрейму **new_users** выяснили следующую информацию:
- Датафрейм содержит 61733 строки и 4 столбца
- Пропусков и полных дубликатов не найдено
- Выполнили преобразование типов object -> datetime64 в столбце ['first_date']
- Даты регистрации соответствуют ТЗ.

### events

In [ ]:
df_review(events)

In [ ]:
# Посмотрим на пропуски в столбце details:
pd.set_option('display.max_rows', None)

In [ ]:
# Выполним необходимые преобразования типов:
events['event_dt'] = pd.to_datetime(events['event_dt'])
new_users.info()

По датафрейму **events** выяснили следующую информацию:
- Датафрейм содержит 440317 строки и 4 столбца
- Дубликаты не были найдены.
- Обнаружены пропуски в столбце events['details'] - 377577 шт.
Пропуски возникли из-за того, что в столбце details информация присутствует только там, где  в столбце event_name содержится purchase. \
Столбец details хранит дополнительные данные о событии. Для покупок, purchase, в этом поле хранится стоимость покупки в долларах. Пропуски никак заполнять не будем. Оставим так, как есть.
- Выполнили преобразование типов object -> datetime64 в столбце events['event_dt']

In [ ]:
purchase = events[events['event_name'] == 'purchase']
purchase.info()
events.info()

Количество значений events['details'](стоимость покупки) = Количеству строк events['event_name'] = 'purchase'
Также, в переменной purchase, где содержится вся информация о событиях с названием purchase нет пропусков, значит:информация присутствует только там, где в столбце event_name содержится purchase

### participants

In [ ]:
df_review(participants)

In [ ]:
participants['ab_test'].unique()

В датафрейме присутствует два названия теста. Из которых нас интересует только 'recommender_system_test'.

### Выводы:
По датафрейму **participants** выяснили следующую информацию:
- Датафрейм содержит 18268 строк и 3 столбца
- Пропусков и полных дубликатов не найдено
- Преобразование типов данных не требуется

## Проверка соответствия данных техническому заданию

### Оценка корректности проведения теста в соответствии с ТЗ

Выделим пользователей, участвующих в тесте и проверим:

- регион регистрации пользователей: все ли попавшие в тест пользователи представляют целевой регион и составляет ли общее количество пользователей из целевого региона 15% от общего числа пользователей из целевого региона, зарегистрированных в период набора пользователей в тест;
- динамику набора пользователей в группы теста и проверьте равномерность распределения пользователей по группам теста и корректность их формирования;

**Название теста: recommender_system_test**\

В датафрейме **participants** содержатся 2 названия тестов: 
- recommender_system_test - который нам нужен, выделим эти строки в отдельную переменную.
- interface_eu_test - оставим эти данные в первоначальной переменной.

In [ ]:
participants.info()

In [ ]:
participants.head(5)

In [ ]:
# Выделяем наш тест:
recommender = participants[participants.ab_test == 'recommender_system_test']
recommender.info()

In [ ]:
recommender.head(5)


In [ ]:
interface = participants[participants.ab_test == 'interface_eu_test']
interface.info()

In [ ]:
# Проверяем: 
recommender['ab_test'].unique()

Замечаем, что количество строк в новом датасете значительно сократилось, \
Остались только те строки, в которых присутствует нужный нам тест 'recommender_system_test', который указан в ТЗ.


Далее, приссоединим к таблице recommender(в которой содержатся нужные нам для исследования пользователи из теста recommender_system_test) таблицу new_users (которая содержит сведения о регистрации, регионе, и платформе):

In [ ]:
recommender = recommender.merge(new_users)
recommender.head(5)

In [ ]:
print('Минимальная дата регистрации', recommender['first_date'].min())
print('Максимальная дата регистрации', recommender['first_date'].max())

Даты регистрации соответствуют периоду, который указан в ТЗ.

Посмотрим на даты событий, которые совершали пользователи:
- все события новых пользователей в период с 7 декабря 2020 по 4 января 2021 года;\

Проверим, так ли это на самом деле:

In [ ]:
print('Минимальная дата событий', events['event_dt'].min())
print('Максимальная дата событий', events['event_dt'].max())

In [ ]:
# Посмотрим,  из каких регионов пользователи нашего теста:
recommender['region'].unique()

Не все попавшие в тест пользователи представляют целевой регион - 'EU'

In [ ]:
# Выделяем пользователей необходимого региона:
eu = recommender[recommender['region'] == 'EU']
len(eu)

Теперь в переменной eu содержатся только те пользователи, которые относятся к необходимому тесту и целевому региону "EU".\
По условию задания, таких пользователей должно быть 15% от общего количества участников, соответствующих тз.

Проверяем: \
составляет ли общее количество пользователей из целевого региона 15% от общего числа пользователей из целевого региона, зарегистрированных в период набора пользователей в тест;

In [ ]:
print('Процент = ', (len(eu)/(len(new_users[new_users['region'] == 'EU'])))*100,'%')


**Выводы:**
- Период набора пользователей в тест соответствует ТЗ
- Не все попавшие в тест пользователи представляют целевой регион - 'EU'
- Количество пользователей целевого региона составляет 15% от общего количества пользователей теста.

- Динамика набора пользователей в группы теста, равномерность распределения пользователей по группам теста и корректность их формирования

In [ ]:
# Сформируем таблицу, в которой отобразим динамику регистраций пользователей каждой из групп:
date_login = (recommender.groupby(['first_date','group'], as_index=False)['user_id'].nunique()
                            .rename(columns={'user_id':'user_cnt'}))
date_login
# Построим график по таблице:

In [ ]:
# Построим график по таблице:

fig = px.bar(date_login, x='first_date', y='user_cnt', color='group', title = 'Динамика набора пользователей в группы А и В исследуемого теста')
fig.show("png")

На гистограмме можем заметить, что новые пользователи регистрировались неравномерно. 9,11 и 17 декабря зарегистрировалось рекондно малое число пользователей. 7, 14,21 декабря наблюдаем пиковые значения. \
Также, можем заметить, что группы А и В формировались неравномерно: набор числа пользователей для группы А значительно преобладает над пользователями группы В.  Можем сделать вывод о том, что группы формировались некорректно еще на этапе регистрации пользователей.

### Пересечения с конкурирующим тестом и пользователи в двух группах теста 

- Удостоверимся, что нет пересечений с конкурирующим тестом и нет пользователей, участвующих в двух группах теста одновременно.
- Удостоверимся, что нет пользователей, участвующих в двух группах теста одновременно.


In [ ]:
# получаем id пользователей, пересечений с конкурирующим тестом
rival=list(np.intersect1d(interface['user_id'], recommender['user_id']))
print(len(rival), 'id пользователей присутствуют в обоих тестах')

print('Доля id пользователей присутствуют в обоих тестах, от общих данных', round(len(rival)/len(participants),2))

In [ ]:
# Удаляем id пользователей, которые попали в обе группы:
recommender = recommender.query('user_id not in @rival')
len(recommender)

Обнаружили пересечения между двумя тестами. Оставить их не можем, т.к. пользователи, участвующие в двух тестах одновременно, могли бы исказить исследуемые данные совершенно непредсказуемым образом. Такие пользователи могут сделать наше исследование некорректным. Потому, мы приняли решение в пользу удаления.

In [ ]:
# Проверяем:
list(np.intersect1d(interface['user_id'], recommender['user_id']))

Проверим пересечение  пользователей, участвующих в двух группах теста одновременно:

In [ ]:
a_rec = recommender[recommender['group'] == 'A']
b_rec = recommender[recommender['group'] == 'B']
print(list(np.intersect1d(a_rec['user_id'], b_rec['user_id'])))

Пересечений между участниками групп А и В нет.

In [ ]:
# распределение пользователей по группам A и B теста recommender_system_test:
recommender['group'].value_counts()


In [ ]:
print('Доля пользователей группы А', round(100*(len(a_rec)/len(recommender))), '%')
print('Доля пользователей группы В', round(100*(len(b_rec)/len(recommender))),'%')

**Выводы:**
- Пересечения с конкурирующим тестом были удалены
- Нет пользователей, участвующих в двух группах теста одновременно
- Доля пользователей группы А 57 %
- Доля пользователей группы В 43 %

### Данные о пользовательской активности:

Изучим:
- даты совершения событий участниками теста: совпадают ли они с датами проведения теста, согласно техническому заданию;
- активность пользователей: все ли зарегистрированные пользователи прошли авторизацию и совершали переход по продуктовой воронке; если есть пользователи, которые не совершали событий после регистрации, изучите их количество и распределение между группами теста; сделайте вывод о необходимости учитывать пользователей без событий при изучении результатов теста;
- горизонт анализа: рассчитайте лайфтайм совершения события пользователем после регистрации, оставьте только те события, которые были совершены в первые 14 дней с момента регистрации; проверьте, что все участники теста имели возможность совершать события все 14 дней с момента регистрации, оцените когда пользователи совершают свои первые события каждого вида.
- Представьте развернутый вывод о соответствии теста требованиям технического задания и возможности получения достоверных результатов АБ-теста, исходя из базового показателя конверсии в 50%.

In [ ]:
print ('Минимальная дата события', events['event_dt'].min())
print ('Максимальная дата события',events['event_dt'].max())

Дата начала проведения теста совпадает с минимальной датой события.
Видим, что макcимальная дата совершенного события - '2020-12-30', что говорит о несоответствии данных тз. 
К сожалению, мы ничего не можем сделать с пропущенными датами, и это, негативно скажется на качестве исследуемых данных: вероятно, мы можем не увидеть полной картины исследования, т.к. отслеживаем эффект спустя 14 дней после регистрации, а действий за последние 5 дней нет.

Объединим таблицы recommender и events, чтобы продолжить работу:

#### Активность пользователей

In [ ]:
recommender_with_nan = recommender.merge(events, how='left', on='user_id').dropna(axis='index', how='any',subset=['group'])
recommender_with_nan.head(5)

In [ ]:
recommender_events = events.merge(recommender, how='left', on='user_id').dropna(axis='index', how='any',subset=['group'])

In [ ]:
recommender_with_nan.info()

In [ ]:
# Уникальные события:
recommender_with_nan['event_name'].unique()

- nan - пользователь не совершал событий после регистрации
- login - пользователь прошел авторизацию
- product_cart - пользователь открыл корзину
- product_page - пользователь заходил в карточку с товаром
- purchase - пользователь совершил покупку

In [ ]:
# Суммарное количество событий, которое совершалось уникальными пользователями
recommender_with_nan['event_name'] = recommender_with_nan['event_name'].fillna('without')
user_events = recommender_with_nan.groupby('event_name').agg({'user_id':'nunique'}).sort_values(by = 'user_id', ascending = False).reset_index()
user_events['share']= round(user_events['user_id']/(len(recommender_with_nan['user_id'].unique())),2)

user_events

Соответственно, только 55% всех зарегистрированных пользователей прошли авторизацию. Далее, по мере продвижения по продуктовой воонке, количество пользователей, совершающих те или иные действия, также, сокращается. 

In [ ]:
print('Уникальное количество пользователей, которые совершали какие-либо события',recommender_events['user_id'].nunique())
print('Уникальный процент пользователей, которые совершали какие-либо события',round(recommender_events['user_id'].nunique()/len(recommender),2))

In [ ]:
# Распределение пользователей, которые не совершали действий после регистрации по группам:
without_actions = recommender_with_nan[recommender_with_nan['event_name'] == 'without']
without_actions['group'].value_counts()
print('Количество пользователей, которые не совершали действий после регистрации в группе А:', round(len(without_actions[without_actions['group'] == 'A'])/len(without_actions['user_id']),2))
print('Количество пользователей, которые не совершали действий после регистрации в группе В:', round(len(without_actions[without_actions['group'] == 'B'])/len(without_actions['user_id']),2))

**Пользователи, которые не совершали события после регистрации:**
- Количество таких пользователей: 2311 или 45%
- Пользователи без событий в группе А: 821 / 36%
- Пользователи без событий в группе В: 1490 / 64%
- Пользователи распределны по группам неравномерно.
- Далее по проекту с пользователями, которые не совершали действий, мы работать не будем. Таких пользователей достаточно много. Однако, мы мало заинтересованы в их изучении,т.к. они малоинформативны. Кроме того, если оставить таких пользователей для дальнейшего анализа, то на этапе составления продуктовой воронки мы получим чрезвычайно низкие значения конверсии(при переходе между различными этапами воронки). 

#### Горизонт анализа
- рассчитайте лайфтайм совершения события пользователем после регистрации
- оставьте только те события, которые были совершены в первые 14 дней с момента регистрации; 
- проверьте, что все участники теста имели возможность совершать события все 14 дней с момента регистрации, оцените когда пользователи совершают свои первые события каждого вида.

будем использовать датафрейм recommender_events, в котором нет пользователей, которые не совершали действия:

In [ ]:
recommender_events['lifetime'] = (recommender_events['event_dt'] - recommender_events['first_date']).dt.days
print('Строк до фильтрации:', len(recommender_events))
recommender_events = recommender_events[(recommender_events['lifetime']<=14)]
print('Строк после фильтрации:',len(recommender_events))

In [ ]:
print('Лайфтайм:', recommender_events.lifetime.nunique())
print( recommender_events.lifetime.value_counts())

In [ ]:
recommender_events.head(5)

In [ ]:
a = recommender_events.groupby('event_name').agg({'lifetime':'mean'})
a

Пользователи совершают свои первые события каждого вида, обычно в трехдневный период после регистрации.

In [ ]:
recommender_events['end_lifetime'] = recommender_events['first_date'] + timedelta(days=14)
_ = recommender_events[recommender_events['end_lifetime'] < '2020-12-31']['user_id'].nunique()
_

In [ ]:
#Всего пользователей:
print(_, 'из', recommender_events['user_id'].nunique(),'Пользователей имели возможность совершать события все 14 дней' )


С учётом результатом п.1, а также результатов анализа lifetime, мы можем сделать вывод о том, что ждать 14 дней с момента регистрации для оценки конверсии по пользователю не нужно, тк пользователи совершают свои первые события каждого вида, обычно в трехдневный период после регистрации.

### Выводы о соответствии данных ТЗ:

- Период набора пользователей в тест соответствует ТЗ
- Количество пользователей целевого региона составляет 15% от общего количества пользователей теста.
- Пересечения с конкурирующим тестом были удалены.
- После удаления пользователи, участвующие в групах А и В единовременно отсутсвуют.
- Пользователи распределены неравномерно между группами:
    - Доля пользователей группы А 57 %
    - Доля пользователей группы В 43 %
    
**Даты совершения событий участниками теста:**
- Даты совершения событий участниками теста совпадают с требованиями ТЗ лишь отчасти. 

- Фактически:
    - Минимальная дата событий 2020-12-07 00:00:33
    - Максимальная дата событий 2020-12-30 23:36:33
    
- По условиям ТЗ:
    - Дата запуска: 2020-12-07
    - Дата остановки: 2021-01-04
Имеем данные о событиях на 5 дней меньше, чем указано в ТЗ.\
Нехватка данных может негативно сказаться на результатах АВ-тестирования.

**Активность пользователей:**
- Пользователей, которые совершали какие-либо события 55% от общего числа. Из них:
    - в группе А: 36%
    - в группе В: 64%
    
**Горизонт анализа**
- Пользователи совершают свои первые события каждого вида, обычно в пределах трехдневного периода после регистрации.

**Соответствие теста требованиям ТЗ:**

При проверке данных были выявлены определенные нарушения, из-за которых следует настороженно относиться к результатам теста и их интерпретации

In [ ]:
print('Количество пользователей, в группе А:', recommender_events[recommender_events['group'] == 'A']['user_id'].nunique())
print('Количество пользователей, в группе В:', recommender_events[recommender_events['group'] == 'B']['user_id'].nunique())

**Возможность получения достоверных результатов АБ-теста, исходя из базового показателя конверсии в 50:%**
Воспользовавшись  калькулятором АВ-тестов, мы выяснили, что при
- Базовом показателе конверсии 50%
- Ожидаемом эффекте улучшения каждой метрики не менее, чем на 5%
- Статистической значимости 5%
- Уровне статистической можности 80%
**Размер выборки для каждого варианта должен быть равен 1,567**

Вывод:
- Аудитория в группе А является достаточной  для получения статистически достоверных результатов нашего теста.
- Размер выборки в группе В не является достаточным.

## Исследовательский анализ данных:

- Распределение количества событий на пользователя в разрезе групп теста: постройте гистограмму распределения этой величины в разрезе групп и сравните её средние значения между собой у групп теста;
- Динамика количества событий в группах теста по дням: изучите распределение числа событий по дням и сравните динамику групп теста между собой.
- Убедитесь, что время проведения теста не совпадает с маркетинговыми и другими активностями. Настройте автоматическую проверку, выдающую список событий, пересекающихся с тестом. При необходимости оцените воздействие маркетинговых событий на динамику количества событий.
- Продуктовая воронка: постройте простые продуктовые воронки для двух групп теста с учетом логической последовательности совершения событий; изучите изменение конверсии в продуктовой воронке тестовой группы, по сравнению с контрольной: наблюдается ли ожидаемый эффект увеличения конверсии в группе В, относительно конверсии в группе А?
- Сделайте общий вывод об изменении пользовательской активности в тестовой группе, по сравнению с контрольной.

### Количество событий на пользователя

- Распределение количества событий на пользователя в разрезе групп теста: 


In [ ]:
print('Доля пользователей группы А', round(100*(recommender_events.query('group == "A"')['event_name'].count()/len(recommender_events))), '%')
print('Доля пользователей группы В', round(100*(recommender_events.query('group == "B"')['event_name'].count()/len(recommender_events))), '%')

In [ ]:
user_a = round (recommender_events.query('group == "A"')['event_name'].count() / recommender_events
                .query('group == "A"')['user_id'].nunique(), 1)
user_b = round (recommender_events.query('group == "B"')['event_name'].count() / recommender_events.
                query('group == "B"')['user_id'].nunique(), 1)


In [ ]:
print ('Среднее количество событий на пользователя в группе А = ',user_a)
print ('Среднее количество событий на пользователя в группе B = ',user_b)

Распределение количества событий между группами А и В неравномерно.
- Среднее количество событий на пользователя в группе А =  6.9
- Среднее количество событий на пользователя в группе B =  5.4
Среди событий наиболее популярное - авторизация - 'login'. Меньшую популярность имеют события "product_cart", "purchase". Данные события совершало наименьшее количество уникальных пользователей.

In [ ]:
mean_ = pd.pivot_table(recommender_events, index = ['user_id', 'group'], values = ['event_name'], aggfunc = 'count').reset_index()


mean_a = mean_.query('group == "A"').groupby(['event_name', 'group']).nunique().sort_values(by = 'event_name', ascending = False).reset_index()

mean_a

In [ ]:
mean_b = mean_.query('group == "B"').groupby(['event_name', 'group']).nunique().sort_values(by = 'event_name', ascending = False).reset_index()
mean_b

In [ ]:
fig = make_subplots(rows=2, cols=1,
                    subplot_titles=("Пользователей группы А:", "Пользователей группы В:"))


fig.append_trace(go.Bar(
    x=mean_a.event_name,
    y=mean_a.user_id,
    text = mean_a.user_id,
    name='A'
), row=1, col=1)

fig.append_trace(go.Bar(
    x=mean_b.event_name,
    y=mean_b.user_id,
    text = mean_b.user_id,
    name='B',
), row=2, col=1)

# Update xaxis properties
fig.update_xaxes(title_text="Количество событий, совершенных одним пользователем", row=1, col=1)
fig.update_xaxes(title_text="Количество событий, совершенных одним пользователем", range=[0, 25], row=2, col=1)


# Update yaxis properties
fig.update_yaxes(title_text="Количество пользователей", row=1, col=1)
fig.update_yaxes(title_text="Количество пользователей", range=[0, 500], row=2, col=1)


fig.update_layout(height=1000, width=1000,
                  title_text="Частота совершенных событий для...")

fig.show("png")


In [ ]:
users_events = (recommender_events.groupby(['user_id', 'event_name'], as_index=False)['region'].count()
                              .rename(columns={'region':'events_count'})
                              .merge(recommender_events[['user_id', 'group']], on='user_id')
                              .drop_duplicates())

In [ ]:
fig = px.box(users_events, x="event_name", y="events_count", color="group")
fig.update_layout(title="Распределение среднего количества событий на пользователя",
                  yaxis_title="количество событий на пользователя",
                  xaxis_title="События")

fig.show("png")

In [ ]:
mean = (recommender_events.groupby(['user_id'], as_index=False)['region'].count()
                              .merge(recommender_events[['user_id', 'group']], on='user_id')
                              .drop_duplicates())
mean

In [ ]:
fig = px.box(mean, y="region", color="group")
fig.update_layout(title="Распределение среднего количества событий на пользователя",
                  yaxis_title="количество событий на пользователя",
                  xaxis_title="Группа")
fig.show("png")

Распределение количества событий на пользователя в разрезе групп теста показало значительное отставание в группе В.

### Количество событий в группах теста по дням

Динамика количества событий в группах теста по дням: 
- изучите распределение числа событий по дням и сравните динамику групп теста между собой.

In [ ]:
recommender_events.info()

In [ ]:
event_per_day = pd.pivot_table(recommender_events, index = ['event_dt', 'group'], values = 'user_id', aggfunc = 'count').reset_index()
event_per_day['event_dt'] =  event_per_day['event_dt'].dt.date
event_per_day = event_per_day.groupby(['event_dt', 'group'])['user_id'].sum().reset_index()
event_per_day.head(5)

In [ ]:
fig = px.line(event_per_day, x="event_dt", y="user_id", color='group', 
              title='Динамика количества событий в группах теста по дням')
fig.show("png")

Динамика групп теста по дням показала значительное отставание по количеству действий группы В.

### Пересечение времени проведения теста и других активностей

- Убедитесь, что время проведения теста не совпадает с маркетинговыми и другими активностями. 
- Настройте автоматическую проверку, выдающую список событий, пересекающихся с тестом. 
- При необходимости оцените воздействие маркетинговых событий на динамику количества событий.

Время теса проведения совпадает с маркетинговыми событиями.

In [ ]:
# Проверим, проводились ли какие-либо маркетинговые события в период проведения АВ теста:
marketing_events[marketing_events['start_dt']>'2020-12-07']

Период проведения АВ теста совпал с проведением Новогодних маркетинговых событий:
- Christmas&New Year Promo	
- CIS New Year Gift Lottery	

Пересечение маркетинговых компаний, с АВ-тестом могло существенно сказаться на ключевых метриках и исказить результаты теста.
Событие "CIS New Year Gift Lottery" не повлияло на наш тест, т.к. его начало совпало с окончанием данных, имеющихся у нас.

Посмотрим количество  уникальных пользователей, авторизировавшихся в период проведения нашего теста:

Не все активности релевантны для нас с учетом региона.\
Наш тест ориентирован на пользователей из Европы.\
С географией и датой нашего теста совпадает маркетинговое событие "Christmas&New Year Promo".\
По графику динамики количества событий, а также по графику авторизировавшихся пользователей мы не наблюдаем никаких аномальных активностей, даже наоборот. С 21 декабря активностиь пользователей начинает стремительно снижаться. И, даже с началом маректинговой кампании тренд к снижению активности продолжается.\
Следовательно, маркетинговое событие, вероятно, не оказало сильного влияния на активность пользователей.

### Продуктовая воронка
- постройте простые продуктовые воронки для двух групп теста с учетом логической последовательности совершения событий; 
- изучите изменение конверсии в продуктовой воронке тестовой группы, по сравнению с контрольной: 
наблюдается ли ожидаемый эффект увеличения конверсии в группе В, относительно конверсии в группе А?

In [ ]:
total = recommender_events['user_id'].nunique()
funnel_table = recommender_events.groupby('event_name').agg({'user_id':'nunique'}).reindex(
['login', 'product_page', 'product_cart','purchase']).reset_index()
funnel_table['n/first '] = 100*round(funnel_table['user_id']/funnel_table['user_id'].max(),4)
funnel_table['n/n-1'] = 100*round(funnel_table['user_id']/funnel_table['user_id'].shift(1),4)
funnel_table

**Рассуждаем о том, почему `purchase` больше, чем `product_cart`:**
Покупок меньше, чем просмотротров корзин, вероятно, потому что пользователи могут совершать покупки не заходя в корзину, а прямо со страницы товара.
Если пользователю  необходимо купить только лишь один конкретный товар, то он может сделать это очень быстро, не заходя в корзину.\
Соответственно, в том случае, если пользователь покупает несколько товаров сразу, что он вынужден воспользоваться переходом на страницу корзины.

Либо же, пользщователь по умолчанию, в любой момент может переидти к оплате всей корзины, непосредственно не заходя в нее. 

In [ ]:
fig = px.funnel(funnel_table, y='event_name', x='user_id',
                template="simple_white",
                title='Продуктовая воронка',
                labels={"Stage": ""})
fig.show("png")

In [ ]:
funnel_group = pd.pivot_table(recommender_events, index = ['event_name', 'group'], values = 'user_id', aggfunc = 'nunique').reset_index()
funnel_group = funnel_group.reindex([0,1,4,5,2,3, 6,7])
funnel_group

In [ ]:
fig = px.funnel(funnel_group, y='event_name', x='user_id', color='group',
                template="simple_white",
                title='Продуктовая воронка групп А/В',
                labels={"Stage": ""})
fig.show("png")

**Конверсия по группам:**

In [ ]:
print('Конверсия пользователей группы А, просмотр корзины:',
      round((funnel_group.loc[2, 'user_id']/funnel_group.loc[0,'user_id'])*100), '%')
print('Конверсия пользователей группы B, просмотр корзины:',
      round((funnel_group.loc[3, 'user_id']/funnel_group.loc[1,'user_id'])*100), '%')

In [ ]:
print('Конверсия пользователей группы А, открывших карточку товара:',
      round((funnel_group.loc[4, 'user_id']/funnel_group.loc[0,'user_id'])*100), '%')
print('Конверсия пользователей группы B, открывших карточку товара:',
      round((funnel_group.loc[5, 'user_id']/funnel_group.loc[1,'user_id'])*100), '%')

In [ ]:
print('Конверсия пользователей группы А, совершивших покупку:',
      round((funnel_group.loc[6, 'user_id']/funnel_group.loc[0,'user_id'])*100), '%')
print('Конверсия пользователей группы B, совершивших покупку:',
      round((funnel_group.loc[7, 'user_id']/funnel_group.loc[1,'user_id'])*100), '%')

Ожидаемый эффект увеличения конверсии в группе В, относительно конверсии в группе А не наблюдается.
Конверсия всех событий в группе А выигрывает по сравнению с группой В.

### Выводы по исследовательскому анализу данных

- Общий вывод об изменении пользовательской активности в тестовой группе, по сравнению с контрольной.

**Количество событий на уникального пользователя в разрезе групп теста распределено неравномерно:**

- Среднее количество событий на пользователя в группе А =  6.9
- Среднее количество событий на пользователя в группе B =  5.4
__________



**Динамика количества событий в группах теста по дням:**

На всей продолжительности АВ теста группа В демонстрирует нам меньшее количество событий в день, по сравнению с группой А.
__________


**Время проведения теста совпадает с маркетинговыми и другими активностями**

Период проведения АВ теста совпал с проведением Новогодних маркетинговых событий. Событие "Christmas&New Year Promo" может сказаться на результатах АВ тестирования, так как частично совпадает с датой его проведения, а именно с 2020-12-25 по 2021-01-03.

----------

**Продуктовая воронка:**

Ожидаемый эффект увеличения конверсии в группе В, относительно конверсии в группе А не наблюдается. Конверсия всех событий в группе А выигрывает по сравнению с группой В.
__________


## Оценка результатов A/B-тестирования:

### Проверка статистической разницы долей z-критерием.

Проверка гипотезы о наличии различий между конверсиями в группах А и В:

- Н1: Между конверсиями групп А и В статистически значимая разница отсутствует.

- Н0: Есть статистически значимая разница между конверсиями групп А и В.

In [ ]:
def conversion(target, login):    
    alpha = 0.05 
    p1 = target[0] / login[0]
    p2 = target[1] / login[1]
    p_combined = (target[0] + target[1]) / (login[0] + login[1]) 
    difference = p1 - p2
    z_value = difference / mth.sqrt(p_combined * (1 - p_combined) * (1 / login[0] + 1 / login[1]))                                               
    distr = st.norm(0, 1) # задаем стандартное нормальное распределение
    p_value = (1 - distr.cdf(abs(z_value))) * 2
    
    if (p_value < alpha):
        print('Отвергаем нулевую гипотезу: между группами есть статистически значимые различия.')
        
    else:
        print('Не получилось отвергнуть нулевую гипотезу, нет статистически значимых различий между группами.')
       
    return print('p_value:', p_value)

**Проверка гипотезы о равнестве конверсий в группах.\
Целевое событие - просмотр корзины (product_cart)**

In [ ]:
login = np.array([funnel_group.loc[0, 'user_id'], funnel_group.loc[1, 'user_id']])
target = np.array([funnel_group.loc[2, 'user_id'], funnel_group.loc[3, 'user_id']])
conversion(target, login)

**Проверка гипотезы о равнестве конверсий в группах.\
Целевое событие - открытие карточки товара (product_page)**

In [ ]:
target = np.array([funnel_group.loc[4, 'user_id'], funnel_group.loc[5, 'user_id']])
conversion(target, login)

**Проверка гипотезы о равнестве конверсий в группах.\
Целевое событие - совершение покупки (purchase)**

In [ ]:

target = np.array([funnel_group.loc[6, 'user_id'], funnel_group.loc[7, 'user_id']])
conversion(target, login)

**При проверке гипотез о равенстве конверсий, выяснили следующее:**
- Конверсия целевых событий "product_cart" и "product_cart" не имеют статистически значимых различий между группами А и В.
- Конверсия целевого события "product_page" имеет статистически значимую разницу между группами А и В. Было зафиксировано преимущество контрольной группы относительно тестовой.

### Выводы по оценке результатов А/B-тестирования

Ожидаемый эффект в изменении конверсии достигнут не был.
Ожидаемый эффект в ТЗ: за 14 дней с момента регистрации в системе пользователи покажут улучшение каждой метрики не менее, чем на 5 процентных пунктов.


## Выводы
Опишите выводы по этапу исследовательского анализа данных и по проведённой оценке результатов A/B-тестирования. Сделайте общее заключение о корректности проведения теста. Дайте рекомендации.

По результатам проверки текущего АВ-теста, были выявлены следующие несоответствия:
- Дата остановки теста по ТЗ: 2021-01-04, что не соответствует фактически полученным данным из таблицы 
поступление информации закончилось ранее 04 января, а именно: остановка набора данных (файл events) была зафиксирована 2020-12-30. 
- пользователи были распределены на группы неравномерно: 2082/705(79%/21%) на этапе авторизации
- Не были достигнуты ожидаемые показатели конверсии. В тестовой группе, по сравнению с контрольной, показатели либо не изменились, либо ухудшились.
- данное А\В тестирование пересекается с маркетинговыми акциями и новогодним периодом, что автоматически делает исследование некорректным.
- Проведение нескольких, пересекающихся между собой АВ-тестирваний недопустимо.
- Задействование одних и тех же уникальных пользователей в разных АВ-тестах недопустимо.


**Рекомендации:**
Не стоит доверять результатам текущего АВ теста.
Основываясь на полученных результатах исследования, рекомендуется перезапусть АВ тест, с учетом следующих рекомендаций:
- Не должно быть пересечений с какими-либо маркетинговыми событиями и промо-акциями, также стоит обратить внимение на близлежащие календарными праздники. Провести АВ тест таким образом, чтобы он не пересекался с ними.
- Проследить за равномерностью распределения участников в обеих группах.
- Контролировать полноту предоставления данных. Выявить причину отсутствия части данных в предыдущем АВ тесте.